# Stochastic Partial Derivative Equations

<!-- SUMMARY: Estimation and Simulations performed in the framework of SPDE -->

<!-- CATEGORY: SPDE -->

In this tutorial, we show how to use the SPDE. We compare some calculations performed *by hand* with the results obtained through gstlearn API interfaces. Note that we also consider (probably temporarily) the Old interface (instantiating the SPDE class and performing subsequent calculations within this class) and the new interface where individual functions are designed for Kriging, Simulating and calculating LogLikelihood.

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import numpy as np
import matplotlib.pyplot as plt

from sksparse.cholmod import cholesky
import scipy as sc
from scipy.sparse import *
from scipy.sparse.linalg import *
import numpy as np

gdoc.setNoScroll()

## Parameters

In [ ]:
# Data
np.random.seed(123)
ndat = 1000

# Model
rangev = 0.2
sill = 1.
nugget = 0.1

# Grid 
nx = [50,50]
dx = [0.02,0.02]
x0 = [0,0]

#Grid meshing
nxm = [75,75]
dxm = [0.02,0.02]
x0m = [-0.25,-0.25]

dbfmt = gl.DbStringFormat.createFromFlags(flag_stats=True, names=["spde*"])

### Grid and Meshing

In [ ]:
grid = gl.DbGrid.create(nx,dx,x0)
gridExt = gl.DbGrid.create(nxm,dxm,x0m)
mesh = gl.MeshETurbo(gridExt)

### Model

In [ ]:
model = gl.Model.createFromParam(gl.ECov.MATERN,param=1,range=rangev,sill=sill)
model.addCovFromParam(gl.ECov.NUGGET,sill=nugget)
model.setDriftIRF()

In [ ]:
model

### Data

In [ ]:
dat = gl.Db.create()
dat["x"] = np.random.uniform(size=ndat)
dat["y"] = np.random.uniform(size=ndat)
dat.setLocators(["x","y"],gl.ELoc.X)

## SPDE non-conditional simulation

### Grid query

In [ ]:
spde = gl.SPDE(model,grid,None,gl.ESPDECalcMode.SIMUNONCOND)
gl.law_set_random_seed(131351)
iuid = spde.compute(grid)
grid.display(dbfmt)

In [ ]:
gp.init(figsize=[7,7],flagEqual=True)
gp.raster(grid)
gp.decoration(title="Non Conditional Simulation")

### Data query

In [ ]:
gl.law_set_random_seed(131351)
iuid = spde.compute(dat)
dat.display(dbfmt)

## Kriging

In [ ]:
spdeRes = gl.SPDE(model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh,1)
iuid = spdeRes.compute(grid)
grid.display(dbfmt)

In [ ]:
gp.init(figsize=[7,7],flagEqual=True)
gp.raster(grid)
gp.symbol(dat, c='black')
gp.decoration(title="Estimation")

## Manually

### Projection Matrix: mesh to grid

In [ ]:
Pglg = gl.ProjMatrix(grid,mesh)
Aprojg = Pglg.toTL()

### Simulation

In [ ]:
Q = spdeRes.getPrecisionOpMatrix().getQ().toTL()
cholQ = cholesky(Q)

In [ ]:
u = np.random.normal(size = Q.shape[0])
gridExt["simuManual"] = cholQ.apply_Pt(cholQ.solve_Lt(1./np.sqrt(cholQ.D())*u))
gridExt.addSelection((gridExt["x1"]>0) & (gridExt["x2"]>0) & (gridExt["x1"]<1.) & (gridExt["x2"]<1.))

gp.init(figsize=[7,7],flagEqual=True)
gp.raster(gridExt, "simuManual", useSel=False)
gp.decoration(title="Simulation (manual)")
print(f"Variance = {round(np.var(gridExt['simuManual'][np.where(gridExt['NewSel']==1)]),4)}")

### Kriging

In [ ]:
Pgl = gl.ProjMatrix(dat,mesh)
Aproj = Pgl.toTL()

Qop = Q + 1/nugget * Aproj.T @ Aproj
cholQop =  cholesky(Qop)

kriging = cholQop.solve_A(Aproj.T @ (dat["spde*"]/nugget))
grid["manually"] = Aprojg @ kriging

In [ ]:
ax = plt.scatter(grid["manually"],grid["*estim"],s=1)
p = plt.plot([-3,3],[-3,3],c="r")

## Likelihood

Manually with Cholesky vs. matrix-free approach with SPDE api.

In [ ]:
def solveMat(cholQop,x):
    return cholQop.solve_A(x)

def invSigma(sigma2,Aproj,cholQop,x):
    return 1./sigma2 * (x - 1./sigma2 * Aproj @ solveMat(cholQop, Aproj.T @ x))

def detQ(cholQ):
    return cholQ.logdet()

x = dat["spde"]
ones = np.ones_like(x)
invSigmaOnes = invSigma(nugget,Aproj,cholQop,ones)
mu  = np.sum(x * invSigmaOnes) / np.sum(ones * invSigmaOnes) 
nMC = 100
print(f"Value for MU = {round(mu,4)}")

In [ ]:
a_quad = np.sum((x-mu)*invSigma(nugget,Aproj,cholQop,x-mu))
b_quad = spdeRes.computeQuad()
print(f"Quadratic (manual)  = {round(a_quad,4)}")
print(f"Quadratic (api-old) = {round(b_quad,4)}")
print(f"-> Relative difference quadratic = {round(100*(b_quad - a_quad) / a_quad,2)}%")

In [ ]:
a_op = detQ(cholQop)
b_op = spdeRes.getPrecisionKrig().computeLogDetOp(1)
print(f"log_det_op (manual)  = {round(a_op,4)}")
print(f"log_det_op (api-old) = {round(b_op,4)}")
print(f"-> Relative difference = {round(100*(b_op-a_op)/a_op, 2)}%")

In [ ]:
a_one = detQ(cholQ)
b_one = spdeRes.getPrecisionKrig().computeLogDetQ(10)
print(f"log_det_Q (manual)  = {round(a_one,4)}")
print(f"log_det_Q (api-old) = {round(b_one,4)}")
print(f"-> Relative difference = {round(100*(b_one-a_one)/a_one,2)}%")

### Comparing the different outputs

- Manual calculation

In [ ]:
logdetnoise = len(x) * np.log(nugget)
logdet = a_op - a_one + logdetnoise
a = -0.5 * (a_quad + logdet + len(x) * np.log(2. * np.pi))
print("Likelihood calculation (manual):")
print(f"log_det_op      = {round(a_op,4)}")
print(f"log_det_Q       = {round(a_one,4)}")
print(f"log_det_Noise   = {round(logdetnoise,4)}")
print(f"log_determinant = {round(logdet,4)}")
print(f"Quadratic term  = {round(a_quad,4)}")
print(f"-> Likelihood (manual) = {round(a,4)}")

- Using the old API 

In [ ]:
spdeLL = gl.SPDE(model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh,1)
b = spdeLL.computeLogLikelihood(nMC, verbose=True)
print(f"-> likelihood (api-old) = {round(b,4)}")

- Using the new API (we use the same 'mesh' as for the manual case to obtain the same results).

In [ ]:
useCholesky = 1
params = gl.SPDEParam.create(nMC=nMC)
meshes = [mesh]
b2 = gl.logLikelihoodSPDE(dat,model,useCholesky=useCholesky, meshes=meshes, params=params, verbose=True)
print(f"-> likelihood (api-new) cholesky=1 {round(b2,4)}")

In [ ]:
useCholesky = 0
b2 = gl.logLikelihoodSPDE(dat,model,useCholesky=useCholesky, meshes=meshes, params=params, verbose=True)
print(f"-> likelihood by New API with cholesky=0 {round(b2,4)}")